
<p align="center">
    <img src="https://github.com/GeostatsGuy/GeostatsPy/blob/master/TCG_color_logo.png?raw=true" width="220" height="240" />

</p>

## Subsurface Data Analytics 

## Interactive Demonstration of Machine Learning Norms

#### Michael Pyrcz, Associate Professor, University of Texas at Austin 

##### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1) | [GeostatsPy](https://github.com/GeostatsGuy/GeostatsPy)


### PGE 383 Exercise: Interactive Demonstration of Machine Learning Norms

Here's a simple workflow, demonstration of predictive machine learning model norms. We use a:

* linear regression model

* 1 preditor feature and 1 response feature

for an high interpretability model/ simple illustration.

#### Norms

Given a vector of error over the $n_{train}$ training data.

\begin{equation}
\Delta y \rightarrow \Delta y_i, \forall i = 1,\dots,n_{train}
\end{equation}

We require a summarization of the error as a single value, this is a norm.

A norm has the following properties:

* norm of a vector maps the vaector values to a summary measures $\rightarrow [0,\infty)$

Common norms include, Manhattan, Euclidean and the general p-norm.

**Manhattan Norm** is defined as:

\begin{equation}
||\Delta y||_1 = \sum_{i=1}^{n_{train}} |\Delta y_i| 
\end{equation}

**Euclidean Norm** is defined as:

\begin{equation}
||\Delta y||_2 = \sqrt{ \sum_{i=1}^{n_{train}} \left( \Delta y_i \right)^2 }
\end{equation}

**p-Norm** is defined as:

\begin{equation}
||\Delta y||_p = \left( \sum_{i=1}^{n_{train}} \left( \Delta y_i \right)^p \right)^{\frac{1}{p}}
\end{equation}

#### Workflow Goals

Learn the basics of machine learning training, tuning for model generalization while avoiding model overfit.

This includes:

* Demonstrate model training and tuning by hand with an interactive exercies

* Demonstrate the role of data error in leading to model overfit with complicated models

#### Import Required Packages

We will also need some standard packages. These should have been installed with Anaconda 3.

In [1]:
%matplotlib inline
import sys                                              # supress output to screen for interactive variogram modeling
import io
import numpy as np                                      # arrays and matrix math
import pandas as pd                                     # DataFrames
import matplotlib.pyplot as plt                         # plotting
from scipy.optimize import minimize                     # linear regression training by-hand with variable norms
from ipywidgets import interactive                      # widgets and interactivity
from ipywidgets import widgets                            
from ipywidgets import Layout
from ipywidgets import Label
from ipywidgets import VBox, HBox

#### Declare Functions

We have functions to perform linear regression for any norm. The code was modified from [N. Wouda](https://stackoverflow.com/questions/51883058/l1-norm-instead-of-l2-norm-for-cost-function-in-regression-model).
* I modified the original functions for a general p-norm linear regression method


In [2]:
def predict(X, params):                                 # linear prediction
    return X.dot(params)

def loss_function(params, X, y, p):                     # custom p-norm, linear regression cost function
    return np.sum(np.power(np.abs(y - predict(X, params)),p))

#### Interactive Dashboard

This code designed the interactive dashboard, prediction model and plots

In [5]:
# widgets and dashboard
l = widgets.Text(value='                                       Machine Learning Norms Demo, Prof. Michael Pyrcz, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))

p_norm = widgets.FloatSlider(min=0.1, max = 10, value=1.0, step = 0.2, description = '$L^{p}$',orientation='horizontal', style = {'description_width': 'initial'}, continuous_update=False)
n = widgets.IntSlider(min=15, max = 80, value=30, step = 1, description = 'n',orientation='horizontal', style = {'description_width': 'initial'}, continuous_update=False)
std = widgets.FloatSlider(min=0.0, max = .95, value=0.00, step = 0.05, description = 'Error (St.Dev.)',orientation='horizontal',style = {'description_width': 'initial'}, continuous_update=False)
xn = widgets.FloatSlider(min=0, max = 1.0, value=0.5, step = 0.05, description = '$X_{n+1}$',orientation='horizontal',style = {'description_width': 'initial'}, continuous_update=False)
yn = widgets.FloatSlider(min=0, max = 1.0, value=0.5, step = 0.05, description = '$Y_{n+1}$',orientation='horizontal', style = {'description_width': 'initial'}, continuous_update=False)

ui1 = widgets.HBox([p_norm,n,std],)
ui2 = widgets.HBox([xn,yn],)
ui = widgets.VBox([l,ui1,ui2],)

def run_plot(p_norm,n,std,xn,yn):                       # make data, fit models and plot

    np.random.seed(73073)                               # set random number seed for repeatable results

    X_seq = np.linspace(0,100.0,1000)                   # make data and add noise
    X_seq = np.asarray([np.ones((len(X_seq),)), X_seq]).T
    X = np.random.rand(n)*0.5
    y = X*X + 0.0 # fit a parabola
    y = y + np.random.normal(loc = 0.0,scale=std,size=n) # add noise
    X = np.asarray([np.ones((n,)), X]).T                 # concatenate a vector of 1's for the constant term
    
    X = np.vstack([X,[1,xn]]); y = np.append(y,yn)       # add the user specified data value to X and y
    
    x0 = [0.5,0.5]                                       # initial guess of model parameters
    p = 2.0
    output_l2 = minimize(loss_function, x0, args=(X, y, p)) # train the L2 norm linear regression model
    p = 1.0
    output_l1 = minimize(loss_function, x0, args=(X, y, p)) # train the L1 norm linear regression model
    p = 3.0
    output_l3 = minimize(loss_function, x0, args=(X, y, p)) # train the L3 norm linear regression model
    
    p = p_norm
    output_lcust = minimize(loss_function, x0, args=(X, y, p)) # train the p-norm linear regression model

    y_hat_l1 = predict(X_seq, output_l1.x)               # predict over the range of X for all models
    y_hat_l2 = predict(X_seq, output_l2.x)
    y_hat_l3 = predict(X_seq, output_l3.x)
    y_hat_lcust = predict(X_seq, output_lcust.x)
    
    plt.subplot(111)                                     # plot the results
    plt.scatter(X[:(n-1),1],y[:(n-1)],s=20,facecolor = 'yellow', edgecolor = 'black', alpha = 0.4)
    plt.scatter(X[n,1],y[n],s=40,marker='^',facecolor = 'orange', edgecolor = 'black', alpha = 0.4)
    plt.plot(X_seq[:,1],y_hat_l1,c = 'blue',alpha = 0.3,label = "L1 Norm")
    plt.plot(X_seq[:,1],y_hat_l2,c = 'red',alpha = 0.3,label = "L2 Norm")
    plt.plot(X_seq[:,1],y_hat_l3,c = 'green',alpha = 0.3,label = "L3 Norm")
    plt.plot(X_seq[:,1],y_hat_lcust,c = 'black',alpha = 1.0,label = "L"+ str(p_norm) + " Norm")
    plt.xlabel(r'Predictor Feature, $X_{1}$'); plt.ylabel(r'Response Feature, $y$'); plt.title('Linear Regression with Various Norms')
    plt.xlim([0.0,1.0]); plt.ylim([0.0,1.0])
    plt.legend(loc = 'lower right')
    
    plt.subplots_adjust(left=0.0, bottom=0.0, right=1.5, top=1.6, wspace=0.9, hspace=0.3)
    plt.show()
    
# connect the function to make the samples and plot to the widgets    
interactive_plot = widgets.interactive_output(run_plot, {'p_norm':p_norm,'n':n,'std':std,'xn':xn,'yn':yn})
interactive_plot.clear_output(wait = True)               # reduce flickering by delaying plot updating

### Interactive Machine Learning Norms Demonstation 

#### Michael Pyrcz, Associate Professor, The University of Texas at Austin 

Observe the impact of choice of norm with variable number of sample data, the data noise, and an outlier! 

### The Inputs

* **p-norm** - 1 = Manhattan norm, 2 = Euclidean norm, etc., **n** - number of data, **Error** - random error in standard deviations
* **$x_{n+1}$**, **$y_{n+1}$** - x and y location of an additional data value

In [6]:
display(ui, interactive_plot)                           # display the interactive plot

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<Figure size 432x288 with 1 Axes>', 'i…

#### Comments

This was a basic demonstration of machining learning norms. I have many other demonstrations and even basics of working with DataFrames, ndarrays, univariate statistics, plotting data, declustering, data transformations and many other workflows available at https://github.com/GeostatsGuy/PythonNumericalDemos and https://github.com/GeostatsGuy/GeostatsPy. 
  
#### The Author:

### Michael Pyrcz, Associate Professor, University of Texas at Austin 
*Novel Data Analytics, Geostatistics and Machine Learning Subsurface Solutions*

With over 17 years of experience in subsurface consulting, research and development, Michael has returned to academia driven by his passion for teaching and enthusiasm for enhancing engineers' and geoscientists' impact in subsurface resource development. 

For more about Michael check out these links:

#### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)

#### Want to Work Together?

I hope this content is helpful to those that want to learn more about subsurface modeling, data analytics and machine learning. Students and working professionals are welcome to participate.

* Want to invite me to visit your company for training, mentoring, project review, workflow design and / or consulting? I'd be happy to drop by and work with you! 

* Interested in partnering, supporting my graduate student research or my Subsurface Data Analytics and Machine Learning consortium (co-PIs including Profs. Foster, Torres-Verdin and van Oort)? My research combines data analytics, stochastic modeling and machine learning theory with practice to develop novel methods and workflows to add value. We are solving challenging subsurface problems!

* I can be reached at mpyrcz@austin.utexas.edu.

I'm always happy to discuss,

*Michael*

Michael Pyrcz, Ph.D., P.Eng. Associate Professor The Hildebrand Department of Petroleum and Geosystems Engineering, Bureau of Economic Geology, The Jackson School of Geosciences, The University of Texas at Austin

#### More Resources Available at: [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)  
  